# Steps Analysis (02/17/17 - 04/10/19)

In [27]:
from app import FXYRS
from stats_functions_class import MainStats
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import pandas as pd
init_notebook_mode(connected=True)

In [28]:
fxyrs = FXYRS('../data/export.xml')
floors_df, dist_df, steps_df, combined_df = fxyrs.clean()

In [29]:
combined_df.head()

,start_date,end_date,num_steps,tot_dist,num_floors,ft_per_step
0,2017-02-17,2017-02-17,812.0,0.342537,1.0,2.227332
1,2017-02-18,2017-02-18,2668.0,1.048047,1.0,2.074097
2,2017-02-19,2017-02-19,2991.0,1.249733,3.0,2.206148
3,2017-02-20,2017-02-20,2812.0,1.178545,0.0,2.212915
4,2017-02-21,2017-02-21,2819.0,1.110353,1.0,2.079696


In [30]:
Msw = MainStats(combined_df, 'week', 'distance')
week_df = Msw.custom_stats()
week_df.dropna(inplace=True)
week_df.sort_values(by=['week_range'], inplace=True)
week_df.loc[:, 'week_range'] = week_df['week_range'].str.replace('201', '1').str.replace(' to ', '-')
week_df.loc[:, 'week_range'] = week_df['week_range'].str.split('-')
week_df.loc[:, 'week_range'] = week_df['week_range'].apply(lambda x: f'{x[1]}.{x[2]}.{x[0]}')

In [31]:
Msm = MainStats(combined_df, 'month', 'distance')
month_df = Msm.custom_stats()
month_df.dropna(inplace=True)
month_df.loc[:, 'month_year'] = pd.to_datetime(month_df.month_year, infer_datetime_format=False)
month_df.sort_values(by=['month_year'], inplace=True)
month_df.loc[:, 'month_year'] = month_df.month_year.dt.strftime('%B %Y')

In [32]:
Msdw = MainStats(combined_df, 'day of week', 'distance')
dow_df = Msdw.custom_stats()
dow_df.dropna(inplace=True)
dow_df.loc[:, 'weekday_year'] = pd.to_datetime(dow_df.weekday_year, infer_datetime_format=False)
dow_df.sort_values(by=['weekday_year'], inplace=True)
dow_df.loc[:, 'weekday_year'] = dow_df.weekday_year.dt.strftime('%A %Y').str.replace('day', '').str.replace('nes', '').str.replace('Satur', 'Sat')

In [35]:
def ldf(x):
    return x.split('-')

In [36]:
Msdm = MainStats(combined_df, 'day of month', 'distance')
dom_df = Msdm.custom_stats()
dom_df.dropna(inplace=True)
dom_df.loc[:, 'date_year'] = dom_df.date_year.apply(lambda x: f"{ldf(x)[1]}-{ldf(x)[0]}")
dom_df.loc[:, 'date_year'] = dom_df.date_year.astype(str)
dom_df.sort_values(by=['date_year'], inplace=True)

In [37]:
smx = week_df.steps_score.max()
dmx = week_df.dist_score.max()
fmx = week_df.floors_score.max()
week_df.loc[:, 'steps_score'] = (week_df.steps_score / smx) * 100
week_df.loc[:, 'dist_score'] = (week_df.dist_score / dmx) * 100
week_df.loc[:, 'floors_score'] = (week_df.floors_score / fmx) * 100
for col in week_df.columns:
    if 'step' in col or 'score' in col:
        week_df.loc[:, col] = round(week_df[col], 1)
week_df.reset_index(inplace=True)
week_df.drop(columns=['index'], inplace=True)
week_df.loc[:, 'mean_dist'] = round(week_df['mean_dist'], 2)
week_df.loc[:, 'mean_flrs'] = round(week_df['mean_flrs'], 2)
week_df.tail(50)

,week_range,std_steps,std_dist,std_flrs,std_fps,mean_steps,mean_dist,mean_flrs,mean_fps,steps_score,dist_score,floors_score
63,04.29.18,2311.5,1.008256,1.496026,0.099150,6055.1,2.59,1.29,2.248897,18.3,19.3,1.3
64,05.06.18,714.5,0.313041,2.160247,0.101529,6164.9,2.62,3.00,2.248597,61.5,63.7,5.0
65,05.13.18,1848.0,0.718123,2.544836,0.107614,8322.6,3.65,1.86,2.321868,43.3,53.6,1.6
66,05.20.18,1465.1,0.642013,2.225395,0.053032,8697.4,3.82,2.43,2.323180,59.7,66.0,3.2
67,05.27.18,4351.4,1.976038,19.633548,0.064123,11831.7,5.14,15.14,2.278350,37.2,38.6,14.1
68,06.03.18,2399.1,1.123941,13.087617,0.069579,5815.6,2.67,19.43,2.413182,16.3,18.4,34.7
69,06.10.18,2801.7,1.320700,10.177005,0.055854,7628.1,3.42,22.29,2.355608,24.0,25.6,58.7
70,06.17.18,809.1,0.393667,10.056981,0.064707,6615.3,3.01,18.86,2.401518,62.5,66.6,42.5
71,06.24.18,2322.1,1.088997,7.889112,0.121476,8246.1,3.58,17.29,2.284287,33.8,34.2,45.6
72,07.01.18,3230.5,1.445694,12.858201,0.076297,7380.4,3.23,20.00,2.310722,19.5,20.9,37.4


In [38]:
b_ws = go.Bar(x=week_df.week_range, y=week_df.mean_steps,
              name='Daily Average Steps', yaxis='y1')

sl_ws = go.Scatter(x=week_df.week_range, y=week_df.steps_score,
                        name='Steps Score', yaxis='y2')

l_ws = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black', 'font': {'color': 'white'},
                  'title': 'Weekly Steps and Score',
                  'legend': {'orientation': 'h', 'x': .32, 'y': 1.1},
                  'yaxis': {'title': 'Daily Average Steps', 'type': 'linear'},
                  'yaxis2': {'title': 'Steps Score', 'titlefont': {'color': 'orange'},
                             'tickfont': {'color': 'orange'}, 'overlaying': 'y', 'side': 'right'}})

d_ws = [b_ws, sl_ws]
f_ws = go.Figure(data=d_ws, layout=l_ws)
iplot(f_ws)

In [39]:
bars = go.Bar(x=week_df.week_range, y=week_df.mean_dist,
              name='Avg Distance Walked (Miles)', yaxis='y1')

score_line = go.Scatter(x=week_df.week_range, y=week_df.dist_score,
                        name='Distance Walked Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black',
                    'font': {'color': 'white'},
                    'title': 'Weekly Distance Walked (Miles) and Score',
                    'legend': {'orientation': 'h', 'x': .25, 'y': 1.1},
                    'yaxis': {'title': 'Avg Distance Walked (Miles)'},
                    'yaxis2': {'title': 'Distance Walked Score',
                               'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'},
                               'overlaying': 'y',
                               'side': 'right'},
                   'xaxis': {'type': '-'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [40]:
bars = go.Bar(x=week_df.week_range, y=week_df.mean_flrs,
              name='Avg Floors Climbed', yaxis='y1')

score_line = go.Scatter(x=week_df.week_range, y=week_df.floors_score,
                        name='Floors Climbed Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black',
                    'font': {'color': 'white'},
                    'title': 'Weekly Floors Climbed and Score',
                    'legend': {'orientation': 'h', 'x': .29, 'y': 1.1},
                    'yaxis': {'title': 'Avg Floors Climbed'},
                    'yaxis2': {'title': 'Floors Climbed Score',
                               'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'},
                               'overlaying': 'y',
                               'side': 'right'},
                   'xaxis': {'type': '-'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [41]:
smx = month_df.steps_score.max()
dmx = month_df.dist_score.max()
fmx = month_df.floors_score.max()
month_df.loc[:, 'steps_score'] = (month_df.steps_score / smx) * 100
month_df.loc[:, 'dist_score'] = (month_df.dist_score / dmx) * 100
month_df.loc[:, 'floors_score'] = (month_df.floors_score / fmx) * 100
for col in month_df.columns:
    if 'step' in col or 'score' in col:
        month_df.loc[:, col] = round(month_df[col], 1)
month_df.reset_index(inplace=True)
month_df.drop(columns=['index'], inplace=True)
month_df.loc[:, 'mean_dist'] = round(month_df['mean_dist'], 2)
month_df.loc[:, 'mean_flrs'] = round(month_df['mean_flrs'], 2)
month_df

,month_year,std_steps,std_dist,std_flrs,std_fps,mean_steps,mean_dist,mean_flrs,mean_fps,steps_score,dist_score,floors_score
0,February 2017,1212.7,0.488778,0.996205,0.078517,2813.7,1.15,1.58,2.168360,12.6,13.5,5.6
1,March 2017,1158.6,0.474636,1.436094,0.076537,3263.1,1.30,2.06,2.096349,17.8,17.7,6.6
2,April 2017,1567.7,0.635312,2.227312,0.046277,3880.3,1.54,3.27,2.083054,18.6,18.5,10.7
3,May 2017,2225.5,0.929945,2.276103,0.086712,3285.8,1.32,2.77,2.107240,9.4,9.3,7.6
4,June 2017,4615.3,1.970746,6.008041,0.069340,6418.2,2.67,5.80,2.177634,17.3,18.0,12.5
5,July 2017,4270.9,1.798718,5.629416,0.064923,5750.5,2.40,5.90,2.193130,15.0,15.9,13.8
6,August 2017,1826.2,0.774469,4.306534,0.073768,4210.7,1.73,3.71,2.161478,18.8,19.2,7.1
7,September 2017,3650.5,1.559112,3.155650,0.079200,5992.3,2.54,4.62,2.227284,19.1,20.6,15.1
8,October 2017,1799.1,0.782296,4.448752,0.099379,7443.2,3.13,8.52,2.224607,59.7,62.4,36.4
9,November 2017,2000.7,0.844048,2.155986,0.063810,7710.7,3.16,7.20,2.163578,57.6,58.9,53.7


In [42]:
bars = go.Bar(x=month_df.month_year, y=month_df.mean_steps,
              name='Avg Steps', yaxis='y1')

score_line = go.Scatter(x=month_df.month_year, y=month_df.steps_score,
                        name='Steps Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black', 'font': {'color': 'white'},
                    'title': 'Monthly Steps and Score', 'yaxis': {'title': 'Avg Steps'},
                    'legend': {'orientation': 'h', 'x': .36, 'y': 1.1},
                    'yaxis2': {'title': 'Steps Score', 'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'}, 'overlaying': 'y',
                               'side': 'right'},
                   'xaxis': {'type': '-'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [43]:
bars = go.Bar(x=month_df.month_year, y=month_df.mean_dist,
              name='Avg Distance Walked (Miles)', yaxis='y1')

score_line = go.Scatter(x=month_df.month_year, y=month_df.dist_score,
                        name='Distance Walked Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black',
                    'font': {'color': 'white'},
                    'title': 'Monthly Distance Walked (Miles) and Score',
                    'legend': {'orientation': 'h', 'x': .25, 'y': 1.1},
                    'yaxis': {'title': 'Avg Distance Walked (Miles)'},
                    'yaxis2': {'title': 'Distance Walked Score',
                               'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'},
                               'overlaying': 'y',
                               'side': 'right'},
                   'xaxis': {'type': '-'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [44]:
bars = go.Bar(x=month_df.month_year, y=month_df.mean_flrs,
              name='Avg Floors Climbed', yaxis='y1')

score_line = go.Scatter(x=month_df.month_year, y=month_df.floors_score,
                        name='Floors Climbed Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black',
                    'font': {'color': 'white'},
                    'title': 'Monthly Floors Climbed and Score',
                    'legend': {'orientation': 'h', 'x': .29, 'y': 1.1},
                    'yaxis': {'title': 'Avg Floors Climbed'},
                    'yaxis2': {'title': 'Floors Climbed Score',
                               'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'},
                               'overlaying': 'y',
                               'side': 'right'},
                   'xaxis': {'type': '-'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [45]:
smx = dow_df.steps_score.max()
dmx = dow_df.dist_score.max()
fmx = dow_df.floors_score.max()
dow_df.loc[:, 'steps_score'] = (dow_df.steps_score / smx) * 100
dow_df.loc[:, 'dist_score'] = (dow_df.dist_score / dmx) * 100
dow_df.loc[:, 'floors_score'] = (dow_df.floors_score / fmx) * 100
for col in dow_df.columns:
    if 'step' in col or 'score' in col:
        dow_df.loc[:, col] = round(dow_df[col], 1)
dow_df.reset_index(inplace=True)
dow_df.drop(columns=['index'], inplace=True)
dow_df.loc[:, 'mean_dist'] = round(dow_df['mean_dist'], 2)
dow_df.loc[:, 'mean_flrs'] = round(dow_df['mean_flrs'], 2)
dow_df

,weekday_year,std_steps,std_dist,std_flrs,std_fps,mean_steps,mean_dist,mean_flrs,mean_fps,steps_score,dist_score,floors_score
0,Sun 2017,3142.0,1.363528,3.936943,0.105460,5619.5,2.35,5.48,2.182081,27.9,28.3,72.3
1,Mon 2017,3418.8,1.489077,6.806266,0.104075,5239.4,2.19,5.24,2.170891,22.3,22.5,38.3
2,Tues 2017,2634.8,1.148584,3.385501,0.097599,4777.7,1.99,4.64,2.165090,24.0,24.0,60.4
3,Wed 2017,3208.6,1.361848,3.594749,0.083961,5062.3,2.09,4.38,2.153439,22.2,22.3,50.5
4,Thurs 2017,3677.7,1.589331,4.849034,0.095470,5277.2,2.18,4.62,2.155201,21.0,20.9,41.8
5,Fri 2017,3292.0,1.419172,4.999798,0.084931,6368.4,2.65,5.96,2.175314,34.2,34.6,67.3
6,Sat 2017,3724.5,1.600540,5.144295,0.094089,7233.2,3.02,6.26,2.187581,39.0,39.7,72.2
7,Mon 2018,2382.8,1.072297,11.733587,0.174713,7869.2,3.26,9.47,2.172631,72.1,69.3,72.5
8,Tues 2018,2348.9,1.024515,8.334336,0.175969,7875.0,3.25,6.60,2.167848,73.3,71.8,49.5
9,Wed 2018,3159.4,1.398046,10.959888,0.172111,7983.9,3.29,9.19,2.157965,56.0,54.2,73.1


In [46]:
bars = go.Bar(x=dow_df.weekday_year, y=dow_df.mean_steps,
              name='Avg Steps', yaxis='y1')

score_line = go.Scatter(x=dow_df.weekday_year, y=dow_df.steps_score,
                        name='Steps Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black', 'font': {'color': 'white'},
                    'title': 'Steps and Score by the Day of the Week ', 'yaxis': {'title': 'Avg Steps'},
                    'legend': {'orientation': 'h', 'x': .35, 'y': 1.1},
                    'yaxis2': {'title': 'Steps Score', 'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'}, 'overlaying': 'y',
                               'side': 'right'},
                   'xaxis': {'type': '-'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [47]:
bars = go.Bar(x=dow_df.weekday_year, y=dow_df.mean_dist,
              name='Avg Distance Walked (Miles)', yaxis='y1')

score_line = go.Scatter(x=dow_df.weekday_year, y=dow_df.dist_score,
                        name='Distance Walked Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black',
                    'font': {'color': 'white'},
                    'legend': {'orientation': 'h', 'x': .26, 'y': 1.1},
                    'title': 'Distance Walked (Miles) and Score by the Day of the Week ',
                    'yaxis': {'title': 'Avg Distance Walked (Miles)'},
                    'yaxis2': {'title': 'Distance Walked Score',
                               'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'},
                               'overlaying': 'y',
                               'side': 'right'},
                   'xaxis': {'type': '-'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [48]:
bars = go.Bar(x=dow_df.weekday_year, y=dow_df.mean_flrs,
              name='Avg Floors Climbed', yaxis='y1')

score_line = go.Scatter(x=dow_df.weekday_year, y=dow_df.floors_score,
                        name='Floors Climbed Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black',
                    'font': {'color': 'white'},
                    'legend': {'orientation': 'h', 'x': .29, 'y': 1.1},
                    'title': 'Floors Climbed and Score by the Day of the Week ',
                    'yaxis': {'title': 'Avg Floors Climbed'},
                    'yaxis2': {'title': 'Floors Climbed Score',
                               'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'},
                               'overlaying': 'y',
                               'side': 'right'},
                   'xaxis': {'type': '-'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [49]:
smx = dom_df.steps_score.max()
dmx = dom_df.dist_score.max()
fmx = dom_df.floors_score.max()
dom_df.loc[:, 'steps_score'] = (dom_df.steps_score / smx) * 100
dom_df.loc[:, 'dist_score'] = (dom_df.dist_score / dmx) * 100
dom_df.loc[:, 'floors_score'] = (dom_df.floors_score / fmx) * 100
for col in dom_df.columns:
    if 'step' in col or 'score' in col:
        dom_df.loc[:, col] = round(dom_df[col], 1)
dom_df.reset_index(inplace=True)
dom_df.drop(columns=['index'], inplace=True)
dom_df.loc[:, 'mean_dist'] = round(dom_df['mean_dist'], 2)
dom_df.loc[:, 'mean_flrs'] = round(dom_df['mean_flrs'], 2)
dom_df

,date_year,std_steps,std_dist,std_flrs,std_fps,mean_steps,mean_dist,mean_flrs,mean_fps,steps_score,dist_score,floors_score
0,2017-01,3921.5,1.688883,3.552777,0.110164,5540.2,2.31,4.80,2.179276,4.1,4.1,48.4
1,2017-02,3917.9,1.799826,3.938415,0.111213,6445.0,2.77,5.80,2.215389,5.6,5.5,63.8
2,2017-03,2840.7,1.140148,3.392803,0.088903,4787.6,1.93,4.20,2.135410,4.3,4.3,38.8
3,2017-04,2249.4,0.959095,3.056868,0.084815,4818.2,1.96,4.30,2.128804,5.5,5.2,45.2
4,2017-05,2516.3,1.035214,2.710064,0.060296,4709.0,1.93,4.30,2.158618,4.7,4.7,50.9
5,2017-06,3228.6,1.372742,6.257440,0.052648,4870.1,2.03,6.40,2.188476,3.9,3.9,48.9
6,2017-07,3078.9,1.298669,4.089281,0.093419,4237.2,1.74,4.50,2.133743,3.1,3.0,37.0
7,2017-08,4692.6,1.868851,5.060742,0.051168,5698.3,2.29,4.50,2.110941,3.7,3.6,29.9
8,2017-09,3232.9,1.347071,3.240370,0.097403,5260.3,2.18,5.50,2.168923,4.5,4.6,69.7
9,2017-10,3475.0,1.490712,3.984693,0.085025,5020.6,2.07,4.10,2.135210,3.8,3.7,31.5


In [50]:
bars = go.Bar(x=dom_df['date_year'], y=dom_df.mean_steps,
              name='Avg Steps', yaxis='y1')

score_line = go.Scatter(x=dom_df['date_year'], y=dom_df.steps_score,
                        name='Steps Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black', 'font': {'color': 'white'},
                    'title': 'Steps and Score by the Day of the Month ', 'yaxis': {'title': 'Avg Steps'},
                    'legend': {'orientation': 'h', 'x': .35, 'y': 1.1},
                    'yaxis2': {'title': 'Steps Score', 'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'}, 'overlaying': 'y',
                               'side': 'right'},
                   'xaxis': {'type': 'category'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [51]:
bars = go.Bar(x=dom_df.date_year, y=dom_df.mean_dist,
              name='Avg Distance Walked (Miles)', yaxis='y1')

score_line = go.Scatter(x=dom_df.date_year, y=dom_df.dist_score,
                        name='Distance Walked Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black',
                    'font': {'color': 'white'},
                    'legend': {'orientation': 'h', 'x': .25, 'y': 1.1},
                    'title': 'Distance Walked (Miles) and Score by the Day of the Month ',
                    'yaxis': {'title': 'Avg Distance Walked (Miles)'},
                    'yaxis2': {'title': 'Distance Walked Score',
                               'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'},
                               'overlaying': 'y',
                               'side': 'right'},
                   'xaxis': {'type': 'category'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [52]:
bars = go.Bar(x=dom_df.date_year, y=dom_df.mean_flrs,
              name='Avg Floors Climbed', yaxis='y1')

score_line = go.Scatter(x=dom_df.date_year, y=dom_df.floors_score,
                        name='Floors Climbed Score', yaxis='y2')

layout = go.Layout({'plot_bgcolor': 'black', 'paper_bgcolor': 'black',
                    'font': {'color': 'white'},
                    'legend': {'orientation': 'h', 'x': .29, 'y': 1.1},
                    'title': 'Floors Climbed and Score by the Day of the Month ',
                    'yaxis': {'title': 'Avg Floors Climbed'},
                    'yaxis2': {'title': 'Floors Climbed Score',
                               'titlefont': {'color': 'orange'},
                               'tickfont': {'color': 'orange'},
                               'overlaying': 'y',
                               'side': 'right'},
                   'xaxis': {'type': 'category'}})

data = [bars, score_line]
fig = go.Figure(data=data, layout=layout)
iplot(fig)